In [18]:
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

In [26]:
HORSE    = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGT"
THEILON  = "GTGGTGCTGTCTGCCGCCGACAAGACCAACGT"
DUCK     = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGT"
GOAT     = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGT"

In [35]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 2                 #number of qubits
    assert n > 2, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

################################################################################################################

    amplitude = np.sqrt(1.0/2**(n-2))
    print(amplitude)
    pr=0
    for i, b in enumerate(bitstring):
        pos = i * 4
            
        if b == "1" or b == "C":
            pos += 1
        elif b == "2" or b == "G":
            pos += 2
        elif b == "3" or b == "T":
            pos += 3
        
        desired_vector[pos] = amplitude
        pr=amplitude**2+pr
  
 #         Cambio requerido en el código para corregir error presentado   #
    
    print(n)
    print(desired_vector.size)
    qc_init = Initialize(desired_vector)
    
 ################################################################################################################       
    
    if not inverse:
#         qc.initialize(desired_vector, qr)

        qc.append(qc_init, qr)  #  Cambio requerido en el código para corregir error presentado   #
    
        qc.barrier(qr)
        
    else:
#         qc.initialize(desired_vector, qr).inverse()  #invert the circuit

        inverse_qc_init = qc_init.gates_to_uncompute()  # Cambio requerido en el código para corregir error presentado   #
        qc.append(inverse_qc_init, qr)  # Cambio requerido en el código para corregir error presentado   #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

In [36]:
n = math.ceil(math.log2(len(HORSE))) + 2                 #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_horse     = encode_bitstring(HORSE, qr, cr)
qc_theilon     = encode_bitstring(THEILON, qr, cr)
qc_duck = encode_bitstring(DUCK, qr, cr)
qc_goat = encode_bitstring(GOAT, qr, cr)


circs = {"HORSE": qc_horse, "DUCK": qc_duck, "GOAT": qc_goat, "THEILON": qc_theilon}

0.1767766952966369
7
128

0.1767766952966369
7
128

0.1767766952966369
7
128

0.1767766952966369
7
128



In [29]:
inverse_qc_horse     = encode_bitstring(HORSE,     qr, cr, inverse=True)
inverse_qc_theilon     = encode_bitstring(THEILON,     qr, cr, inverse=True)
inverse_qc_duck = encode_bitstring(DUCK, qr, cr, inverse=True)
inverse_qc_goat = encode_bitstring(GOAT, qr, cr, inverse=True)


inverse_circs = {"HORSE": inverse_qc_horse, "DUCK": inverse_qc_duck, "GOAT": inverse_qc_goat, "THEILON": inverse_qc_theilon}

0.1767766952966369
1.0000000000000002
128

0.1767766952966369
1.0000000000000002
128

0.1767766952966369
1.0000000000000002
128

0.1767766952966369
1.0000000000000002
128



In [39]:
from qiskit import IBMQ, BasicAer

key = "HORSE"       #the name of the code used as key to find similar ones

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
   # if other_key == key:
     #   continue
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)
print(combined_circs[key])

Similarity score of HORSE and HORSE is 1.0
Similarity score of HORSE and DUCK is 0.8792724609375
Similarity score of HORSE and GOAT is 0.8792724609375
Similarity score of HORSE and THEILON is 0.93896484375
[ANSWER] HORSE is most similar to HORSE
          »
q73781_0: »
          »
q73781_1: »
          »
q73781_2: »
          »
q73781_3: »
          »
q73781_4: »
          »
q73781_5: »
          »
q73781_6: »
          »
    c7: 7/»
          »
«          ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q73781_0: ┤0                                                            

In [32]:
from qiskit.providers.ibmq import least_busy
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='uni-antioquia-1', project='qcudea2022-1')
Remote_QProcessor = least_busy(provider.backends(filters=lambda qprocessor: qprocessor.configuration().n_qubits >= 7 
                                       and not qprocessor.configuration().simulator 
                                       and qprocessor.status().operational==True))
print("El computador cuántico menos ocupado es el " + Remote_QProcessor.name())

key = "HORSE"       #the name of the code used as key to find similar ones

# use local simulator
simulator = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
   # if other_key == key:
     #   continue
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key],Remote_QProcessor,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

ibmqfactory.load_account:WARNING:2022-09-06 02:38:00,247: Credentials are already in use. The existing account in the session will be replaced.


El computador cuántico menos ocupado es el ibm_lagos
Similarity score of HORSE and HORSE is 0.011474609375
Similarity score of HORSE and DUCK is 0.0125732421875
Similarity score of HORSE and GOAT is 0.010986328125
Similarity score of HORSE and THEILON is 0.0096435546875
[ANSWER] HORSE is most similar to DUCK
